In [1]:
"""This script processes employee data from a XLSX file and planning data from an Excel file,
generates a summary, and exports the results to a new Excel file.

The script performs the following steps:
1. Sets the locale to Spanish (es_ES.UTF-8).
2. Defines the year and start date for the planning period.
3. Constructs file paths for the input XLSX and Excel files and the output Excel file.
4. Loads employee data from the YAML file.
5. Creates employee information and dates for the planning period.
6. Initializes employees by shifts based on the dates and restrictions.
7. Checks if the planning Excel file exists; if not, exits the script.
8. Reads the planning data from the Excel file.
9. Generates a summary of the total data based on the employees and planning data.
10. Creates a new Excel workbook and removes the default sheet.
11. Exports the planning data for each month to the workbook.
12. Adds the total data summary to the workbook.
13. Saves the workbook to the specified output file.

Modules:
- locale: For setting the locale to Spanish.
- os: For handling file paths.
- sys: For modifying the system path.
- pandas: For reading Excel files.
- openpyxl: For creating and manipulating Excel workbooks.
- planning.config: For employee restrictions configuration.
- planning.employee: For various employee-related functions.

Functions:
- load_employees_from_yaml: Loads employee data from a YAML file.
- create_employees_with_dates: Creates employee information and dates for the planning period.
- init_employees_by_shifts: Initializes employees by shifts based on the dates and restrictions.
- generate_summary_total: Generates a summary of the total data.
- export_month: Exports the planning data for a specific month to the workbook.
- add_total_data: Adds the total data summary to the workbook.
"""
import locale
import os
import sys

import pandas as pd

module_path = os.path.abspath(os.path.join("..", ".."))
sys.path.insert(0, module_path)


from openpyxl import (
    Workbook,
)

from planning.employee import (  # noqa: E402
    add_total_data,
    create_employees_with_dates,
    export_month,
    generate_summary_total,
    init_employees_by_shifts,
    load_config,
    load_employees_from_yaml,
)

locale.setlocale(locale.LC_TIME, "es_ES.UTF-8")

year = 2025
case = "case_1"
start_date = f"{year}-01-01"
script_path = os.path.abspath("")
root_path = os.path.join(script_path, "..", "..", "..")

employees_file = os.path.join(root_path, "data", "2025", case, "employees.yaml")
planning_file = os.path.join(root_path, "data", "2025", case, "data.xlsx")
config_file = os.path.join(root_path, "data", "2025", case, "config.json")
generated_employees_xlsx_from_xlsx = os.path.join(
    root_path, "output", "2025", case,
    "planning_generated_from_xlsx.xlsx"
)

config = load_config(config_file)
employee_restrictions = config["employee_restrictions"]

employees = load_employees_from_yaml(employees_file, employee_restrictions)
employees_info, dates = create_employees_with_dates(start_date, 365, employees)
all_employees_by_shift = init_employees_by_shifts(dates, employee_restrictions)

if not os.path.exists(planning_file):
    print(f"File {planning_file} not found")
    sys.exit(1)
planning_data = pd.read_excel(planning_file, sheet_name=None)
locale.setlocale(locale.LC_TIME, "es_ES.UTF-8")

year = 2025
start_date = f"{year}-01-01"

planning_data = pd.read_excel(planning_file, index_col=0, na_filter=False, sheet_name=None)
total_data = generate_summary_total(employees, employee_restrictions, planning_data)


wb = Workbook()
wb.remove(wb.active)

for month in range(1, 13):
    export_month(wb, f"{month:02d}", planning_data)

add_total_data(wb, total_data)

wb.save(generated_employees_xlsx_from_xlsx)